<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/TRSeminar/blob/main/TRS14_DenseNetwork_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning seminar
## Dr. Carlos Villaseñor
## Dense Neural network in Keras/tensorflow

# Import modules and clean data

Step 1. Import modules

In [1]:
# Paquetería básica
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importamos Tensorflow y Keras
import tensorflow as tf
from tensorflow import keras

# Usamos funcionalidades básicas de Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Step 2. Load dataset: 'auto-mpg.csv' and read it with *Pandas*

In [ ]:
!wget 'https://github.com/Dr-Carlos-Villasenor/Clase_Aprendizaje_Profundo/raw/main/Datasets/auto-mpg.csv'

In [ ]:
dataset = pd.read_csv('auto-mpg.csv')
dataset.head()

Step 3. Explore and clean the dataset

In [ ]:
dataset.info()

In [ ]:
np.array(dataset.horsepower)

In [ ]:
dataset['horsepower'].replace('?', np.nan, inplace=True)
np.array(dataset.horsepower)

In [7]:
dataset = dataset.dropna()

Step 4. Make sure there is not missing data

In [ ]:
dataset.isna().sum()

Step 5. Codify the origin feature using one-hot encoding

In [ ]:
origin = dataset.pop('origin')
dataset['USA'] = (origin == 1) * 1.0
dataset['Europe'] = (origin == 2) * 1.0
dataset['Japan'] = (origin == 3) * 1.0
dataset.tail()

Step 6. Choose the input and output features, scale inputs

In [ ]:
x = np.asanyarray(dataset.drop(columns=['mpg', 'car name']))
y = np.asanyarray(dataset[['mpg']])
x = StandardScaler().fit_transform(x)
print(x.shape)
print(y.shape)

# Train Neural net

Step 7. Split dataset

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

Step 8. Create a Dense Neural network

In [24]:
def build_model():

  model = keras.Sequential()
  model.add(keras.layers.Dense(128, activation='relu', input_shape=[x.shape[1]]))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(32, activation='tanh'))
  model.add(keras.layers.Dense(1, activation='linear'))

  model.compile(loss='mean_squared_error',
                optimizer=keras.optimizers.Adam(),
                metrics=[keras.metrics.MAE, keras.metrics.R2Score()])
  return  model

Step 9. Invoke function and check the model summary

In [ ]:
model = build_model()
model.summary()

Step 10. Train the neural net

In [ ]:
history = model.fit(xtrain, ytrain,
                    batch_size=300, epochs=1300,
                    validation_data=(xtest, ytest), verbose=1)

Step 11. Draw the learining curves in order to diagnosticate the network performance

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  plt.figure(figsize=(12,6))
  plt.subplot(121)
  plt.title('Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(hist['epoch'], hist['loss'],
           label='Train')
  plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val')
  plt.yscale('log')
  plt.legend()

  plt.subplot(122)
  plt.title('R2 Score')
  plt.xlabel('Epoch')
  plt.ylabel('r2score')
  plt.plot(hist['epoch'], hist['r2_score'],
           label='Train')
  plt.plot(hist['epoch'], hist['val_r2_score'],
           label = 'Val')
  plt.ylim([-0.5,1])
  plt.legend()

plot_history(history)

# Save model

Step 12. Save the model

In [41]:
model.save('model.keras')

Step 13. Upload model

In [42]:
model2 = keras.models.load_model('model.keras')